# 06_Model_Evaluation.py

This notebook was automatically converted from a Python script.

In [ ]:

"""
Scikit-learn Model Değerlendirme Teknikleri

Bu script, scikit-learn kütüphanesindeki model değerlendirme yöntemlerini göstermektedir.
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_breast_cancer, load_boston, fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline



Modeller


In [ ]:
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC



Metrikler


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, average_precision_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import warnings
warnings.filterwarnings('ignore')



%matplotlib inline


In [ ]:

print("Scikit-learn Model Değerlendirme Teknikleri")
print("="*50)



1. Sınıflandırma Değerlendirme


In [ ]:
print("\n1. Sınıflandırma Değerlendirme")
print("-"*50)



Meme kanseri veri seti


In [ ]:
cancer = load_breast_cancer()
X_cancer = cancer.data
y_cancer = cancer.target

print("Veri seti boyutu:", X_cancer.shape)
print("Sınıf dağılımı:", np.bincount(y_cancer))
print("Sınıf isimleri:", cancer.target_names)



Eğitim-Test ayırma


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_cancer, y_cancer, test_size=0.3, random_state=42)



Model Pipeline (ölçeklendirme + model)


In [ ]:
clf = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])



Model eğitimi


In [ ]:
clf.fit(X_train, y_train)



Tahminler


In [ ]:
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:, 1]  # Pozitif sınıf olasılıkları



1.1 Karmaşıklık Matrisi (Confusion Matrix)


In [ ]:
print("\n1.1 Karmaşıklık Matrisi (Confusion Matrix)")
print("-"*50)



Karmaşıklık matrisi hesaplama


In [ ]:
cm = confusion_matrix(y_test, y_pred)



Görselleştirme


In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=cancer.target_names,
            yticklabels=cancer.target_names)
plt.xlabel('Tahmin')
plt.ylabel('Gerçek')
plt.title('Karmaşıklık Matrisi')
plt.tight_layout()
plt.savefig('confusion_matrix.png')
plt.close()
print("Karmaşıklık matrisi görselleştirmesi 'confusion_matrix.png' olarak kaydedildi.")



Sınıflandırma raporu


In [ ]:
print("\nSınıflandırma Raporu:")
print(classification_report(y_test, y_pred, target_names=cancer.target_names))



1.2 ROC Eğrisi ve AUC


In [ ]:
print("\n1.2 ROC Eğrisi ve AUC")
print("-"*50)



ROC Eğrisi


In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {auc:.3f}')
plt.plot([0, 1], [0, 1], 'k--', label='Rastgele Tahmin (AUC = 0.5)')
plt.xlabel('Yanlış Pozitif Oranı')
plt.ylabel('Doğru Pozitif Oranı')
plt.title('ROC Eğrisi')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('roc_curve.png')
plt.close()
print("ROC eğrisi görselleştirmesi 'roc_curve.png' olarak kaydedildi.")



1.3 Precision-Recall Eğrisi


In [ ]:
print("\n1.3 Precision-Recall Eğrisi")
print("-"*50)



Precision-Recall Eğrisi


In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
avg_precision = average_precision_score(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'Ortalama Precision = {avg_precision:.3f}')
plt.xlabel('Recall (Duyarlılık)')
plt.ylabel('Precision (Kesinlik)')
plt.title('Precision-Recall Eğrisi')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('precision_recall_curve.png')
plt.close()
print("Precision-Recall eğrisi görselleştirmesi 'precision_recall_curve.png' olarak kaydedildi.")



1.4 Çapraz Doğrulama (Cross-Validation)


In [ ]:
print("\n1.4 Çapraz Doğrulama (Cross-Validation)")
print("-"*50)



K-fold çapraz doğrulama


In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)



Farklı modeller için çapraz doğrulama skoru


In [ ]:
models = {
    'Lojistik Regresyon': Pipeline([('scaler', StandardScaler()),
                                    ('clf', LogisticRegression(max_iter=1000, random_state=42))]),
    'Random Forest': Pipeline([('scaler', StandardScaler()),
                              ('clf', RandomForestClassifier(random_state=42))]),
    'SVM': Pipeline([('scaler', StandardScaler()),
                     ('clf', SVC(probability=True, random_state=42))])
}

cv_results = {}

for name, model in models.items():
    cv_scores = cross_val_score(model, X_cancer, y_cancer, cv=kfold, scoring='accuracy')
    cv_results[name] = cv_scores
    print(f"{name}: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")



Görselleştirme


In [ ]:
plt.figure(figsize=(10, 6))
plt.boxplot([cv_results[name] for name in models.keys()], labels=models.keys())
plt.title('Çapraz Doğrulama Skorları')
plt.ylabel('Doğruluk')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('cross_validation_scores.png')
plt.close()
print("Çapraz doğrulama skorları görselleştirmesi 'cross_validation_scores.png' olarak kaydedildi.")



2. Regresyon Değerlendirme


In [ ]:
print("\n2. Regresyon Değerlendirme")
print("-"*50)



California konut veri seti


In [ ]:
try:
    housing = fetch_california_housing()
    X_housing = housing.data
    y_housing = housing.target
    dataset_name = "California Housing"
except:
    # Boston veri seti alternatifi
    try:
        boston = load_boston()
        X_housing = boston.data
        y_housing = boston.target
        dataset_name = "Boston Housing"
    except:
        # Eğer her ikisi de çalışmazsa, yapay veri oluştur
        from sklearn.datasets import make_regression
        X_housing, y_housing = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
        dataset_name = "Yapay Regresyon"

print(f"{dataset_name} veri seti yüklendi.")
print("Veri seti boyutu:", X_housing.shape)



Eğitim-Test ayırma


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_housing, y_housing, test_size=0.3, random_state=42)



Model Pipeline (ölçeklendirme + model)


In [ ]:
reg = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Ridge(alpha=1.0, random_state=42))
])



Model eğitimi


In [ ]:
reg.fit(X_train, y_train)



Tahminler


In [ ]:
y_pred = reg.predict(X_test)



2.1 Temel Metrikler


In [ ]:
print("\n2.1 Temel Metrikler")
print("-"*50)



MSE, RMSE, MAE ve R² hesaplama


In [ ]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")



Gerçek vs Tahmin grafiği


In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_housing.min(), y_housing.max()], [y_housing.min(), y_housing.max()], 'r--')
plt.xlabel('Gerçek Değerler')
plt.ylabel('Tahmin Edilen Değerler')
plt.title('Gerçek vs Tahmin')
plt.grid(True)
plt.tight_layout()
plt.savefig('true_vs_pred.png')
plt.close()
print("Gerçek vs tahmin değerleri görselleştirmesi 'true_vs_pred.png' olarak kaydedildi.")



Artık (Residual) analizi


In [ ]:
residuals = y_test - y_pred

plt.figure(figsize=(12, 5))

plt.subplot(121)
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Tahmin Edilen Değerler')
plt.ylabel('Artıklar (Residuals)')
plt.title('Artık Analizi')
plt.grid(True)

plt.subplot(122)
plt.hist(residuals, bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Artıklar (Residuals)')
plt.ylabel('Frekans')
plt.title('Artık Dağılımı')
plt.grid(True)

plt.tight_layout()
plt.savefig('residual_analysis.png')
plt.close()
print("Artık analizi görselleştirmesi 'residual_analysis.png' olarak kaydedildi.")



2.2 Regresyon için Çapraz Doğrulama


In [ ]:
print("\n2.2 Regresyon için Çapraz Doğrulama")
print("-"*50)



Farklı regresyon modelleri için çapraz doğrulama skoru


In [ ]:
reg_models = {
    'Ridge': Pipeline([('scaler', StandardScaler()),
                       ('reg', Ridge(alpha=1.0, random_state=42))]),
    'Random Forest': Pipeline([('scaler', StandardScaler()),
                              ('reg', RandomForestRegressor(n_estimators=100, random_state=42))])
}

cv_reg_results = {}

for name, model in reg_models.items():
    mse_scores = -cross_val_score(model, X_housing, y_housing, cv=kfold, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(mse_scores)
    r2_scores = cross_val_score(model, X_housing, y_housing, cv=kfold, scoring='r2')
    
    cv_reg_results[name] = {'RMSE': rmse_scores, 'R²': r2_scores}
    
    print(f"{name}:")
    print(f"  RMSE: {rmse_scores.mean():.4f} (±{rmse_scores.std():.4f})")
    print(f"  R²: {r2_scores.mean():.4f} (±{r2_scores.std():.4f})")



Görselleştirme


In [ ]:
plt.figure(figsize=(14, 6))

plt.subplot(121)
plt.boxplot([cv_reg_results[name]['RMSE'] for name in reg_models.keys()], labels=reg_models.keys())
plt.title('RMSE Çapraz Doğrulama Skorları (Düşük Daha İyi)')
plt.ylabel('RMSE')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.subplot(122)
plt.boxplot([cv_reg_results[name]['R²'] for name in reg_models.keys()], labels=reg_models.keys())
plt.title('R² Çapraz Doğrulama Skorları (Yüksek Daha İyi)')
plt.ylabel('R²')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('regression_cross_validation.png')
plt.close()
print("Regresyon çapraz doğrulama skorları görselleştirmesi 'regression_cross_validation.png' olarak kaydedildi.")



3. Hiperparametre Optimizasyonu


In [ ]:
print("\n3. Hiperparametre Optimizasyonu")
print("-"*50)



Grid Search ile hiperparametre optimizasyonu


Sınıflandırma modeli için


In [ ]:
param_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__solver': ['liblinear', 'saga']
}

grid_search = GridSearchCV(
    estimator=Pipeline([('scaler', StandardScaler()),
                        ('classifier', LogisticRegression(max_iter=1000, random_state=42))]),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)



Grid Search'ü çalıştır


In [ ]:
grid_search.fit(X_cancer, y_cancer)



En iyi parametreler ve skor


In [ ]:
print("En iyi parametreler:", grid_search.best_params_)
print("En iyi çapraz doğrulama skoru:", grid_search.best_score_)



Sonuçların incelenmesi


In [ ]:
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results.sort_values('rank_test_score')

print("\nEn iyi 5 parametre kombinasyonu:")
columns_to_show = [c for c in cv_results.columns if 'param_' in c or 'mean_test_score' in c or 'std_test_score' in c]
print(cv_results[columns_to_show].head(5))



4. Öğrenme Eğrileri (Learning Curves)


In [ ]:
print("\n4. Öğrenme Eğrileri (Learning Curves)")
print("-"*50)



Öğrenme eğrileri - aşırı uyum/düşük uyum tespiti için


In [ ]:
train_sizes, train_scores, test_scores = learning_curve(
    estimator=grid_search.best_estimator_,
    X=X_cancer,
    y=y_cancer,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, 'o-', color='r', label='Eğitim skoru')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='r')
plt.plot(train_sizes, test_mean, 'o-', color='g', label='Çapraz doğrulama skoru')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1, color='g')
plt.xlabel('Eğitim Örnek Sayısı')
plt.ylabel('Doğruluk')
plt.title('Öğrenme Eğrisi')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()
plt.savefig('learning_curve.png')
plt.close()
print("Öğrenme eğrisi görselleştirmesi 'learning_curve.png' olarak kaydedildi.")



5. Validasyon Eğrileri (Validation Curves)


In [ ]:
print("\n5. Validasyon Eğrileri (Validation Curves)")
print("-"*50)



Validasyon eğrileri - hiperparametre etkisi


C parametresi için validasyon eğrisi


In [ ]:
param_range = np.logspace(-3, 3, 7)
train_scores, test_scores = validation_curve(
    estimator=Pipeline([('scaler', StandardScaler()),
                         ('classifier', LogisticRegression(max_iter=1000, random_state=42))]),
    X=X_cancer,
    y=y_cancer,
    param_name="classifier__C",
    param_range=param_range,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.semilogx(param_range, train_mean, 'o-', color='r', label='Eğitim skoru')
plt.fill_between(param_range, train_mean - train_std, train_mean + train_std, alpha=0.1, color='r')
plt.semilogx(param_range, test_mean, 'o-', color='g', label='Çapraz doğrulama skoru')
plt.fill_between(param_range, test_mean - test_std, test_mean + test_std, alpha=0.1, color='g')
plt.xlabel('C Parametresi')
plt.ylabel('Doğruluk')
plt.title('Validasyon Eğrisi')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()
plt.savefig('validation_curve.png')
plt.close()
print("Validasyon eğrisi görselleştirmesi 'validation_curve.png' olarak kaydedildi.") 
